# **GAN** Protein homo

## Required libraries and configuration

In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 22.9 MB/s eta 0:00:00



Import required libraries

In [1]:
import os

import re
import time
import random
import sys

from typing import Dict, Optional, Tuple, List
from collections import OrderedDict

import flwr as fl
from flwr.common import Metrics

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import models, layers, losses, metrics, optimizers

from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter

2023-08-11 09:16:26.301378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-11 09:16:26.301416: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics as skmetrics
from sklearn.metrics import confusion_matrix

In [3]:
from GAN import Gan

Define some parameters for the simulation, such as the number of clients in the federated scenario, the number of federated rounds, the number of epochs of each client before communicating, and the batch size for training phase

In [4]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients in the federated scenario
NUM_ROUNDS = 10 #5 Number of learning rounds in the federated computation
NUM_EPOCHS = 5 #3 Number of epochs that the local dataset is seen each round
BATCH_SIZE = 20 # Batch size for training phase

# Define the seed for random numbers
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)
random.seed(seed)

## Loading and preparing the input data

The Sentiment140 dataset is not available in TFF, so it needs to be loaded from other source such as the tfds (tensorflow datasets) library. Then, it is adapted to the TFF format, so it can be used to train a model using TFF.

Note: We download the full Sentiment140 dataset, but in order to be able to execute the experiments in reasonable time, we are going to use a portion of it. In this notebook we are selecting just 1% of the data for training and 10% for testing purposes.

In [5]:
dset_name = 'protein_homo'
dset = fetch_datasets()[dset_name]

In [6]:
X = pd.DataFrame(StandardScaler().fit_transform(dset.data))
y = dset.target

y[y == -1] = 0
X['label'] = y

dtrain, dtest = train_test_split(X, test_size=0.2, random_state=42, stratify=y)

In [7]:
dtest = dtest.iloc[:-1,:]

In [8]:
dtrain.shape

(116600, 75)

In [9]:
dtrain['label'].value_counts()

0    115563
1      1037
Name: label, dtype: int64

Select the text and polarity columns from the original data, and transform it to a dataframe so it can be later used in TFF.

At this point, we do not select the 'user' column because we will create random IID partitions, so the 'user' column is not neccesary. If the user want to try it with non-IID partition, he/she may also keep the 'user' column and use it as client identifier instead of the following random user ID.

In [10]:
# Create a random list of ids. Each instance is given a random id, indicating the client where will be distributed
ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(dtrain)//NUM_CLIENTS)]
random.Random(seed).shuffle(ids_train)

# Add the id assignment to the dataframe
dtrain['user'] = ids_train
unique_ids_train = list(set(ids_train))

In [11]:
# Do the same with the test data
ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(dtest)//NUM_CLIENTS)]
random.Random(seed+1).shuffle(ids_test)
dtest['user'] = ids_test
unique_ids_test = list(set(ids_test))

For the sake of simplicity, in this notebook we will be dealing with a binary problem. For that purpose, we remove the neutral tweets, so the classifier's aim is to differentiate between positive and negative tweets.

Create the DataLoaders for each client. It creates an array of datasets, one for each client.

In [12]:
# Create DataLoaders for each client
train_data = []
test_data = []
for i in range(NUM_CLIENTS):
    train_data.append(dtrain.loc[dtrain['user']==unique_ids_train[i]])
    test_data.append(dtest.loc[dtest['user']==unique_ids_test[i]])

## Create a Deep Learning model

In this case we use a model composed by a pre-trained model from tfhub, as well as dense layers. The pre-trained model is not updated in the example; however, the ``trainable`` parameter can be set to ``True``, so such layers are also fine-tuned in the collaborative training.

Note that any network architecture supported by keras can be used. Besides, Flower also supports PyTorch models, but in contrast to Use Case 1, we use a TFF model here to provide a variety of options.

In [13]:
def create_keras_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=[74]))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    # Compile the model
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.AUC(from_logits=True)]
    )

    return model

## Training in the federated scenario

First, we create a FlowerClient class, that includes the information of each simulated client. The class has three methods:
 * `get_parameters`: Get the parameters of the model to send them to the server
 * `fit`: Reveives the model parameters from the server, trains it with local data, and return the updated model parameters to the server
 * `evaluate`: Receives the model from the server and evaluates it with local data

In [14]:
def _df(data):
    df = pd.DataFrame(data)
    for c in range(df.shape[1]):
        mapping = {df.columns[c]: c}
        df = df.rename(columns=mapping)
    return df

In [15]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [16]:
# Class-independent methods to get and set the parameters of a model
def get_parameters(model):
        return model.get_weights()

def set_parameters(model, parameters):
    model.set_weights(parameters)

In [17]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, client_train_data, client_test_data):
        # Init the client setting the x (text) and the y (polarity) for both train and testing
        self.model = model

        client_train_data = client_train_data.drop('user', axis=1)
        client_test_data = client_test_data.drop('user', axis=1)

        x_train, y_train = client_train_data.drop('label',axis=1), client_train_data['label'].to_numpy()
        x_test, y_test = client_test_data.drop('label',axis=1), client_test_data['label'].to_numpy()

        # GAN
        X_train_pos = x_train[y_train == 1]

        model_gan = Gan(data=X_train_pos)
        generator = model_gan._generator()
        discriminator = model_gan._discriminator()
        gan_model = model_gan._GAN(generator=generator, discriminator=discriminator)
        trained_model = model_gan.train(generator, discriminator, gan_model, verbose=False)

        # Samples to generate
        sampling_strategy = .7
        num_examples = int((Counter(y_train)[0] - Counter(y_train)[1]) * sampling_strategy)
        noise = np.random.normal(0, 1, (X_train_pos.shape[0] + num_examples, X_train_pos.shape[1]))
        new_data = _df(data=trained_model.predict(noise))

        X_res = np.concatenate((x_train, new_data))
        y_res = np.concatenate((y_train, np.ones(new_data.shape[0])))

        # RUS
        X_eq, y_eq = RUS(X_res, y_res, frac=.02)

        self.x_train, self.y_train = X_eq, y_eq
        self.x_test, self.y_test = x_test, y_test

    def get_properties(self, config):
        """Get properties of client."""
        raise Exception("Not implemented")

    def get_parameters(self, config):
        """Get parameters of the local model."""
        return get_parameters(self.model)

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""
        # Update local model parameters
        set_parameters(self.model, parameters)

        # Get hyperparameters for this round
        batch_size: int = BATCH_SIZE
        epochs: int = NUM_EPOCHS

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size,
            epochs,
            verbose=0
        )

        # Return updated model parameters and results
        parameters_prime = get_parameters(model=self.model)
        num_examples_train = len(self.x_train)
        results = {
            "loss": history.history["loss"][0],
            #"accuracy": history.history["accuracy"][0],

        }
        return parameters_prime, num_examples_train, results

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        set_parameters(self.model, parameters)

        # Get config values
        steps: int = 10

        # Evaluate global model parameters on the local test data and return results
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, 1)
        num_examples_test = len(self.x_test)

        y_pred = self.model.predict(self.x_test)
        y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

        cm = confusion_matrix(self.y_test, y_pred)
        tn = cm[0][0]
        fn = cm[1][0]
        tp = cm[1][1]
        fp = cm[0][1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)

        cr = skmetrics.classification_report(self.y_test, y_pred)
        print(cr)

        return loss, num_examples_test, {"TPR*TNR": tpr*tnr}

To simulate the federated scenario in a single machine, the client_fn method allows to create FlowerClients on demand, given the client id.

Note that each client is passed both training and testing local data, so the evaluation over test data is done during the simulation itself.

In [18]:
def client_fn(cid: str) -> FlowerClient:
    # Load model
    model = create_keras_model()

    # Note: each client gets a different train/test data
    client_train_data = train_data[int(cid)]
    client_test_data = test_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(model, client_train_data, client_test_data)

In order to show averaged evaluations metrics beyond loss, we should define a method to do that; in this case, the accuracy is weighted averaged.

In [19]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["TPR*TNR"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"TPR*TNR": sum(accuracies) / sum(examples)}

Define a training strategy with the weighted FedAvg algorithm.

Then, start the simulation indicating the method to create clients, the number of clients in the simulation, the number of rounds, and the strategy (i.e., the FedAvg strategy to combine local updates). The simulation covers both the federated model training as well as evaluating the model with each local test data.

In [20]:
# Define the configuration during training/fitting the model. It is used later when defining the FedAvg strategy.
def fit_config(server_round: int):
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": NUM_EPOCHS
    }
    return config

# Define the configuration during evaluation. It is used later when defining the FedAvg strategy.
def evaluate_config(server_round: int):
    return {"val_steps": 5}

In [21]:
# Define strategy
strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl_sim = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-08-11 09:17:14,207 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-08-11 09:17:16,455	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-08-11 09:17:17,451 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'node:192.168.1.129': 1.0, 'memory': 22693353063.0, 'object_store_memory': 11346676531.0}
INFO flwr 2023-08-11 09:17:17,452 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-11 09:17:17,454 | server.py:273 | Requesting initial parameters from one random client
(pid=6379) 2023-08-11 09:17:17.693747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=6379) 2023-08-11 09:17:17.693776: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machi

1620/2915 [===============>..............] - ETA: 2s - loss: 0.0596 - auc_1: 0.9723


2835/2915 [============================>.] - ETA: 0s - loss: 0.0419 - auc_1: 0.9984


 778/2915 [=======>......................] - ETA: 4s - loss: 0.0383 - auc_1: 0.9985 [repeated 2x across cluster]
(launch_and_evaluate pid=6388)               precision    recall  f1-score   support
(launch_and_evaluate pid=6388) 
(launch_and_evaluate pid=6388)            0       1.00      0.99      0.99      2892
(launch_and_evaluate pid=6388)            1       0.31      0.83      0.45        23
(launch_and_evaluate pid=6388) 
(launch_and_evaluate pid=6388)     accuracy                           0.98      2915
(launch_and_evaluate pid=6388)    macro avg       0.66      0.91      0.72      2915
(launch_and_evaluate pid=6388) weighted avg       0.99      0.98      0.99      2915
(launch_and_evaluate pid=6388) 
1024/2915 [=========>....................] - ETA: 4s - loss: 0.0339 - auc_1: 0.9988 [repeated 11x across cluster]
(launch_and_evaluate pid=6382) 
1164/2915 [==========>...................] - ETA: 3s - loss: 0.0468 - auc_1: 0.9964 [repeated 3x across cluster]


DEBUG flwr 2023-08-11 09:18:39,841 | server.py:182 | evaluate_round 1 received 10 results and 0 failures


...

2831/2915 [============================>.] - ETA: 0s - loss: 0.0903 - auc_17: 0.9979 [repeated 3x across cluster]
(launch_and_evaluate pid=6385)               precision    recall  f1-score   support [repeated 9x across cluster]
(launch_and_evaluate pid=6385)  [repeated 45x across cluster]
(launch_and_evaluate pid=6385)     accuracy                           0.98      2915 [repeated 9x across cluster]
(launch_and_evaluate pid=6385)    macro avg       0.66      0.99      0.73      2915 [repeated 9x across cluster]
(launch_and_evaluate pid=6385) weighted avg       0.99      0.98      0.98      2915 [repeated 9x across cluster]
  53/2915 [..............................] - ETA: 5s - loss: 0.3888 - auc_19: 0.0000e+00


1856/2915 [==================>...........] - ETA: 2s - loss: 0.1418 - auc_19: 0.9819


2915/2915 [==============================] - 7s 2ms/step - loss: 0.1249 - auc_20: 0.9560
(launch_and_evaluate pid=6389)               precision    recall  f1-score   support
(launch_and_evaluate pid=6389) 
(launch_and_evaluate pid=6389)            0       1.00      0.98      0.99      2889
(launch_and_evaluate pid=6389)            1       0.26      0.92      0.41        26
(launch_and_evaluate pid=6389) 
(launch_and_evaluate pid=6389)     accuracy                           0.98      2915
(launch_and_evaluate pid=6389)    macro avg       0.63      0.95      0.70      2915
(launch_and_evaluate pid=6389) weighted avg       0.99      0.98      0.98      2915
(launch_and_evaluate pid=6389) 
 896/2915 [========>.....................] - ETA: 4s - loss: 0.1397 - auc_19: 0.9961 [repeated 4x across cluster]


DEBUG flwr 2023-08-11 09:28:43,569 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-08-11 09:28:43,570 | server.py:147 | FL finished in 671.6061109490001
INFO flwr 2023-08-11 09:28:43,572 | app.py:218 | app_fit: losses_distributed [(1, 0.045737992972135544), (2, 0.1013031043112278), (3, 0.11096948906779289), (4, 0.12107055783271789), (5, 0.11316843926906586), (6, 0.1163561075925827), (7, 0.11032707244157791), (8, 0.10589220151305198), (9, 0.10549717769026756), (10, 0.10784724205732346)]
INFO flwr 2023-08-11 09:28:43,573 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-11 09:28:43,574 | app.py:220 | app_fit: metrics_distributed {'TPR*TNR': [(1, 0.8747837345777719), (2, 0.9015999214551427), (3, 0.9088759540251673), (4, 0.9119435311174329), (5, 0.9054680425932043), (6, 0.9105288982044241), (7, 0.9116352885025777), (8, 0.9168644000251202), (9, 0.9198461269445134), (10, 0.9058004529932059)]}
INFO flwr 2023-08-11 09:28:43,574 | app.py:

1118/2915 [==========>...................] - ETA: 3s - loss: 0.1281 - auc_19: 0.9938 [repeated 12x across cluster]


## Evaluation with test data

The evaluation has been done during the simulation. Following, we show the averaged results over test data.
The result of the simulation includes the results on all rounds, so we retrieve those of the last round.

In [22]:
print('Test data, \t Loss={:.4f}, \t Accuracy={:.4f}'.format(fl_sim.losses_distributed[-1][1], fl_sim.metrics_distributed['TPR*TNR'][-1][1]))

Test data, 	 Loss=0.1078, 	 Accuracy=0.9058


In [23]:
fl_sim

History (loss, distributed):
	round 1: 0.045737992972135544
	round 2: 0.1013031043112278
	round 3: 0.11096948906779289
	round 4: 0.12107055783271789
	round 5: 0.11316843926906586
	round 6: 0.1163561075925827
	round 7: 0.11032707244157791
	round 8: 0.10589220151305198
	round 9: 0.10549717769026756
	round 10: 0.10784724205732346
History (metrics, distributed, evaluate):
{'TPR*TNR': [(1, 0.8747837345777719), (2, 0.9015999214551427), (3, 0.9088759540251673), (4, 0.9119435311174329), (5, 0.9054680425932043), (6, 0.9105288982044241), (7, 0.9116352885025777), (8, 0.9168644000251202), (9, 0.9198461269445134), (10, 0.9058004529932059)]}